In [ ]:
import arcpy
import os
import censusdata
arcpy.env.overwriteOutput = True

### Cleaning and Clipping Data

In [ ]:
inputFloodShape = r"E:\My Drive\School\College\06 Grad Full\Semester 2\Geospatial Programming\Project 2\Data\FEMA_Floodplain.shp"
outFlood = "Greene_County_MO_FloodZones"

FLD_ZONE = '0.2 PCT ANNUAL CHANCE FLOOD HAZARD' OR FLD_ZONE = 'A' OR FLD_ZONE = 'AE'


<Result 'E:\\My Drive\\School\\College\\06 Grad Full\\Semester 2\\Geospatial Programming\\Project 2\\Default.gdb\\Greene_County_MO_FloodZones'>

In [ ]:
delimitedField = arcpy.AddFieldDelimiters(arcpy.env.workspace, "FLD_ZONE")
expression = delimitedField + " = '0.2 PCT ANNUAL CHANCE FLOOD HAZARD'"
expression += " OR " + delimitedField + " = 'A'"
expression += " OR " + delimitedField + " = 'AE'"

print(expression)

arcpy.FeatureClassToFeatureClass_conversion(inputFloodShape, arcpy.env.workspace, outFlood, expression)

In [ ]:
spatial_ref_flood = arcpy.Describe("Greene_County_MO_FloodZones").spatialReference
print(spatial_ref_flood.name)

GCS_WGS_1984


In [ ]:
Green_Census_location = r"E:\My Drive\School\College\06 Grad Full\Semester 2\Geospatial Programming\Project 2\Data\cb_2018_29_bg_500k.shp"
green_census_block = "Greene_County_MO_Census_Data"

In [ ]:
delimitedField = arcpy.AddFieldDelimiters(arcpy.env.workspace, "COUNTYFP")
expression = delimitedField + " = '077'"
print(expression)

arcpy.FeatureClassToFeatureClass_conversion(Green_Census_location, arcpy.env.workspace, green_census_block, expression)

COUNTYFP = '077'


<Result 'E:\\My Drive\\School\\College\\06 Grad Full\\Semester 2\\Geospatial Programming\\Project 2\\Default.gdb\\Greene_County_MO_Census_Data'>

In [ ]:
#clip the roads to only include ones inside Green County
inputRoadsShp = r"E:\My Drive\School\College\06 Grad Full\Semester 2\Geospatial Programming\Project 2\Data\missouri_highway.shp"
limits = "Greene_County_MO_Census_Data"
outName = "Greene_County_Roads"


<Result 'E:\\My Drive\\School\\College\\06 Grad Full\\Semester 2\\Geospatial Programming\\Project 2\\Default.gdb\\Greene_County_Roads'>

In [ ]:
arcpy.Clip_analysis(inputRoadsShp, limits, outName)

In [ ]:
# find roads that intersect with flood polygons
green_roads = "Greene_County_Roads"
flooded_roads = arcpy.SelectLayerByLocation_management(green_roads, 'INTERSECT', outFlood)

In [ ]:
arcpy.Intersect_analysis([green_roads, outFlood], "roads_intersect_floods")

<Result 'E:\\My Drive\\School\\College\\06 Grad Full\\Semester 2\\Geospatial Programming\\Project 2\\Default.gdb\\roads_intersect_floods'>

### Obtaining Census Data and Joining it with Shapefiles

In [ ]:
data = censusdata.download('acs5', 2019,
           censusdata.censusgeo([('state', '29'),
                                 ('county', '077'),
                                 ('block group', '*')]),
          ['B23025_001E', 'B23025_002E', 'B23025_003E',
           'B23025_004E', 'B23025_005E',
           'B23025_006E', 'B23025_007E'])



data.to_csv(r"E:/My Drive/School/College/06 Grad Full/Semester 2/Geospatial Programming/Project 2/Data/green_county_census_data.csv", index = True)

In [ ]:
##import table to geodatabase
arcpy.TableToTable_conversion(r"E:/My Drive/School/College/06 Grad Full/Semester 2/Geospatial Programming/Project 2/Data/green_county_census_data.csv", arcpy.env.workspace, "greene_census_data")

<Result 'E:\\My Drive\\School\\College\\06 Grad Full\\Semester 2\\Geospatial Programming\\Project 2\\Default.gdb\\greene_census_data'>

In [ ]:
##creating field to join with
censusTableName = "greene_census_data"
arcpy.AddField_management(censusTableName, "geoid", "TEXT")

<Result 'greene_census_data'>

In [ ]:
with arcpy.da.UpdateCursor(censusTableName, ['Field1', 'geoid']) as cursor:
    #update geoid using Field1
    for row in cursor:
        comma = row[0].split(', ')
        greaterThan = comma[-1].split('> ')
#         state = greaterThan[0][-2:]
#         county = greaterThan[1][-3:]
#         tract = greaterThan[2][-6:]
#         blockGroup = greaterThan[3][-1:]
# #         print(state)
# #         print(county)
# #         print(tract)
# #         print(greaterThan)
#         geoid = state + county + tract + blockGroup
#         print(geoid)
        geoid_str = ""
        for i in greaterThan:
            colon = i.split(":")
            geoid_str += colon[1]
        row[1] = geoid_str
        cursor.updateRow(row)

In [ ]:
## Joining census data to Census Shapefile
  ## second table name gets inner joined onto the first one.
greene_cbg_joined = arcpy.AddJoin_management(green_census_block, "GEOID", censusTableName, "geoid")
arcpy.CopyFeatures_management(greene_cbg_joined, "Greene_County_Joined_Census_Data")

<Result 'E:\\My Drive\\School\\College\\06 Grad Full\\Semester 2\\Geospatial Programming\\Project 2\\Default.gdb\\Greene_County_Joined_Census_Data'>

In [ ]:
def exportAttributeJoin(inputFc, idFieldInputFc, inputTable, idFieldTable, workspace):
    
    arcpy.env.workspace = workspace
    outName = inputFc + "_" + inputTable + "_JoinOn_" + idFieldInputFc
    
    joined = arcpy.AddJoin_management(inputFc, idFieldInputFc, inputTable, idFieldTable)
    
    arcpy.CopyFeatures_management(joined, outName)
        
    print("Rows Matched: " + str(arcpy.management.GetCount(joined)))
    print("Input Feature Class Rows: " + str(arcpy.management.GetCount(inputFc)))
    print("Input Table Rows: " + str(arcpy.management.GetCount(inputTable)))

In [ ]:
exportAttributeJoin("Greene_County_MO_Census_Data", "GEOID", "greene_census_data", "geoid", arcpy.env.workspace)

Rows Matched: 167
Input Feature Class Rows: 167
Input Table Rows: 167


### Altering Field Names to run calculations.

In [ ]:
fc = "Greene_County_MO_Census_Data_greene_census_data_JoinOn_GEOID"
newFieldList = [["greene_census_data_B23025_001E", "total_unemployed"],
                ["greene_census_data_B23025_002E", "work_eligible"]]

for field in newFieldList:
    arcpy.AlterField_management(fc, field[0], field[1], field[1])

In [ ]:
## adding an area field
arcpy.AddField_management(fc, "area_sqmi", "DOUBLE")

##calculate area in sqmi
arcpy.CalculateGeometryAttributes_management(fc, [["area_sqmi", "AREA_GEODESIC"]], "MILES_US")

<Result 'Greene_County_MO_Census_Data_greene_census_data_JoinOn_GEOID'>

In [ ]:
## intersect analysis with joined data and flood areas.
arcpy.Intersect_analysis([fc, outFlood], "cbg_intersects_flood")

<Result 'E:\\My Drive\\School\\College\\06 Grad Full\\Semester 2\\Geospatial Programming\\Project 2\\Default.gdb\\cbg_intersects_flood'>

In [ ]:
arcpy.AddField_management("cbg_intersects_flood", "f_area_sqmi", "DOUBLE")
## calculate area of each flood zone in each census block group
arcpy.CalculateGeometryAttributes_management("cbg_intersects_flood", [["f_area_sqmi", "AREA_GEODESIC"]], "MILES_US")

<Result 'cbg_intersects_flood'>

In [ ]:
cbg_inters_f = "cbg_intersects_flood"
flood_area_field = "f_area_sqmi"

cbg_dict = dict()
with arcpy.da.SearchCursor(cbg_inters_f, ["Greene_County_MO_Census_Data_GEOID", flood_area_field]) as cursor:
    for row in cursor:
        geoid = row[0]
        if geoid in cbg_dict.keys():
            cbg_dict[geoid] += row[1]
        else:
            cbg_dict[geoid] = row[1]

In [ ]:
arcpy.AddField_management(fc, flood_area_field, "DOUBLE")

<Result 'Greene_County_MO_Census_Data_greene_census_data_JoinOn_GEOID'>

In [ ]:
with arcpy.da.UpdateCursor(fc, ["Greene_County_MO_Census_Data_GEOID", flood_area_field]) as cursor:
    for row in cursor:
        if row[0] in cbg_dict.keys():
            row[1] = cbg_dict[row[0]]
        else:
            row[1] = 0
        cursor.updateRow(row)

In [ ]:
flood_pct_field = "flood_pct"
arcpy.AddField_management(fc, flood_pct_field, 'DOUBLE')

<Result 'Greene_County_MO_Census_Data_greene_census_data_JoinOn_GEOID'>

In [ ]:
arcpy.CalculateField_management(fc, flood_pct_field, "!f_area_sqmi!/!area_sqmi!", "PYTHON3")

<Result 'Greene_County_MO_Census_Data_greene_census_data_JoinOn_GEOID'>